In [12]:
# merging datasets
import pandas as pd
ptv = pd.read_parquet('../data/curated/final_train_hospital_cbd_dist_data')
shop = pd.read_csv('../data/curated/agg_shopping_centre.csv')
household = pd.read_csv('../data/curated/dwellings_household.csv')
recreation = pd.read_csv('../data/curated/recreation_cleaned.csv')
population = pd.read_csv('../data/curated/total_population.csv')
park_domain = pd.read_parquet('../data/curated/parkres/parkres_domain.parquet')
school = pd.read_csv('../data/curated/house_independent_school.csv')
income = pd.read_parquet('../data/curated/income.parquet')
crime = pd.read_csv('../data/curated/property_data/agg_property_lost.csv')

In [13]:
columns_to_drop = [
    'price', 'address', 'property_type', 'latitude', 'longitude',
    'Beds', 'Baths', 'Parking', 'bond', 'extracted_price', 'geometry',
    'index_right', 'sa2_code', 'sa2_name', 'chg_flag', 'chg_lbl',
    'sa3_code', 'sa3_name', 'sa4_code', 'sa4_name', 'gcc_code', 'gcc_name',
    'ste_code', 'ste_name', 'aus_code', 'aus_name', 'areasqkm', 'loci_uri',
    'geometry_proj', 'Unnamed: 0'
]

# Dropping the columns from the DataFrame
school = school.drop(columns=columns_to_drop)

In [14]:
joint = pd.merge(ptv, shop, on='sa2_code', how='left')
joint = joint.merge(household, left_on='sa2_code', right_on='SA2 code', how='left')
joint = joint.merge(recreation, on='sa2_code', how='left')
joint = joint.merge(population, on='sa2_code', how='left')
joint = joint.merge(park_domain, on='url', how='left')
joint = joint.merge(school, on='url', how='left')
joint = joint.merge(crime,left_on='sa2_name_x', right_on='sa2_name', how='left')
# Drop columns with '_y' suffix
columns_to_drop = [col for col in joint.columns if col.endswith('_y')]
joint_cleaned = joint.drop(columns=columns_to_drop)

# Rename columns with '_x' suffix by removing '_x'
joint_cleaned.columns = [col.replace('_x', '') for col in joint_cleaned.columns]
joint_cleaned['sa2_code'] = joint_cleaned['sa2_code'].astype(str)
income['SA2 code'] = income['SA2 code'].astype(str)
joint_cleaned = joint_cleaned.merge(income, left_on='sa2_code', right_on='SA2 code', how='left')

/var/folders/b6/nv55vzd50gz6bs69664021z00000gn/T/ipykernel_1272/3984844621.py:7: FutureWarning: Passing 'suffixes' which cause duplicate columns {'sa2_name_x'} in the result is deprecated and will raise a MergeError in a future version.
  joint = joint.merge(crime,left_on='sa2_name_x', right_on='sa2_name', how='left')


In [15]:
print(len(joint_cleaned))

9565


In [16]:
joint_cleaned = joint_cleaned.loc[:, ~joint_cleaned.columns.duplicated()]

In [17]:
joint_cleaned.to_parquet('../data/curated/features_domain.parquet')

In [18]:
len(joint_cleaned.columns)

114